In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import csv
import cv2
import data_aug
import importlib

Run with trunk kernel

### PREPROCESSING DATA
Trim dataset for single image set to one x3, y3, z3 set per image

In [21]:
def make_tip_pos_csv(input_csv, output_csv):
    # Check if the output CSV exists and create it with the header if it doesn't
    if not os.path.exists(output_csv):
        header = ['ID', 'x3', 'y3', 'z3', 'img_filename']
        with open(output_csv, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(header)

    # Read the input CSV
    positions_df = pd.read_csv(input_csv)
    
    # Initialize variables to keep track of the last image filename and the last row to write
    last_img_filename = None
    last_output_row = None

    # Process each row in the input CSV
    for index, row in positions_df.iterrows():
        cur_img_filename = row['img_filename']
        img_name = cur_img_filename.removesuffix(".jpg")
        output_row = [row['ID'], row['x3'], row['y3'], row['z3'], cur_img_filename]
        output_row_augmented = [row['ID'], row['x3'], row['y3'], row['z3'], img_name+'_augmented.jpg']

        # If the current image filename is different from the last, write the last output row
        if last_img_filename is not None and last_img_filename != cur_img_filename:
            with open(output_csv, 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(last_output_row) #non-augmented image
                writer.writerow(last_output_row_augmented)# augmented image

        # Update the last image filename and last output row
        last_img_filename = cur_img_filename
        last_output_row = output_row
        last_output_row_augmented = output_row_augmented

    # Write the last row (for the final image in the sequence)
    if last_output_row is not None:
        with open(output_csv, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(last_output_row) # non-augmented image
            writer.writerow(last_output_row_augmented) # augmented image



In [22]:
data_dir = 'data/single'
input_csv = os.path.join(data_dir, 'single_img_regression.csv')  # replace with your actual CSV filename
output_csv = os.path.join(data_dir, 'single_img_regression_single_tip_pos.csv')
make_tip_pos_csv(input_csv, output_csv)

Plot tip position overlaid on images to verify dataset quality

In [61]:
def convert_to_pillow_coords(df, img_width, img_height):
    """
    Convert robot coordinates to Pillow image coordinates.

    Parameters:
    df (pd.DataFrame): DataFrame containing 'x' and 'z' columns in robot coordinates.
    img_width, img_height (int): Dimensions of the Pillow image.

    Returns:
    pd.DataFrame: DataFrame with 'img_x' and 'img_y' columns for Pillow image coordinates.
    """
    buffer = .01 #m

    # Calculate the min and max values for x and z
    x_min, x_max = df['x'].min() - buffer, df['x'].max() + buffer
    z_min, z_max = df['z'].min() - buffer, df['z'].max() + buffer

    # Calculate the scaling factors for x and z coordinates
    x_scale = img_width / (x_max - x_min)
    z_scale = img_height / (z_max - z_min)

    # Calculate the shifts to center the robot's origin within the image
    x_shift = (x_max + x_min) / 2
    z_shift = (z_max + z_min) / 2

    # Convert robot coordinates to Pillow image coordinates
    df['img_x'] = (df['x'] - x_shift) * x_scale + img_width / 2
    df['img_y'] = (df['z'] - z_shift) * z_scale + img_height / 2

    # Invert the y-axis and x-axis to match Pillow's coordinate system (where (0, 0) is top-left)
    df['img_y'] = img_height - df['img_y']
    df['img_x'] = img_width - df['img_x']

    return df[['img_x', 'img_y']]



In [69]:
import matplotlib.pyplot as plt
from PIL import Image

def plot_tip_positions_on_images(data_dir):
    """
    Plot tip positions on images based on coordinates from a CSV file.

    Parameters:
    data_dir (str): Directory containing the CSV file and the images.
    """

    # Load the CSV file
    csv_file = os.path.join(data_dir, 'single_img_regression_single_tip_pos.csv')
    positions_df = pd.read_csv(csv_file)

    # Folder containing images
    image_folder = os.path.join(data_dir, 'images')

    # Convert entire DataFrame coordinates to Pillow image coordinates once
    coords_df = positions_df[['x3', 'z3']].rename(columns={'x3': 'x', 'z3': 'z'})

    # Get image dimensions from the first image
    sample_image_filename = positions_df['img_filename'].iloc[0]
    sample_image_path = os.path.join(image_folder, sample_image_filename)
    with Image.open(sample_image_path) as img:
        img_width, img_height = img.size

    # Convert all coordinates to image coordinates
    img_coords_df = convert_to_pillow_coords(coords_df, img_width, img_height)

    # Add converted coordinates to the original DataFrame
    positions_df['img_x'] = img_coords_df['img_x']
    positions_df['img_y'] = img_coords_df['img_y']

    # Iterate through each row to plot points on the corresponding images
    for index, row in positions_df.iterrows():
        image_filename = row['img_filename']
        image_path = os.path.join(image_folder, image_filename)

        # Open the image
        with Image.open(image_path) as img:
            img_width, img_height = img.size

            # Create a figure with the same dimensions as the image
            fig, ax = plt.subplots(figsize=(img_width / 100, img_height / 100), dpi=100)

            # Plot the image
            ax.imshow(img)

            # Plot the tip position for this specific row
            ax.scatter([row['img_x']], [row['img_y']], color='blue', s=200)  # s is the size of the point

            # Remove axes for a cleaner output
            ax.axis('off')

            # Save the image with the original dimensions
            output_filename = os.path.join(data_dir, f"verification/output_{image_filename}")
            plt.savefig(output_filename, bbox_inches='tight', pad_inches=0)
            plt.close(fig)

            print(f"Plotted tip position on {image_filename} and saved as {output_filename}.")


In [70]:
plot_tip_positions_on_images('data/single')

Plotted tip position on sample_0.jpg and saved as data/single/verification/output_sample_0.jpg.
Plotted tip position on sample_1.jpg and saved as data/single/verification/output_sample_1.jpg.
Plotted tip position on sample_2.jpg and saved as data/single/verification/output_sample_2.jpg.
Plotted tip position on sample_3.jpg and saved as data/single/verification/output_sample_3.jpg.
Plotted tip position on sample_4.jpg and saved as data/single/verification/output_sample_4.jpg.
Plotted tip position on sample_5.jpg and saved as data/single/verification/output_sample_5.jpg.
Plotted tip position on sample_6.jpg and saved as data/single/verification/output_sample_6.jpg.
Plotted tip position on sample_7.jpg and saved as data/single/verification/output_sample_7.jpg.
Plotted tip position on sample_8.jpg and saved as data/single/verification/output_sample_8.jpg.
Plotted tip position on sample_9.jpg and saved as data/single/verification/output_sample_9.jpg.
Plotted tip position on sample_10.jpg an

Augment images

In [26]:
input_dir = 'data/single/raw'
output_dir = 'data/single/augmented'
importlib.reload(data_aug)

for filename in os.listdir(input_dir+"/images"):
    if filename.lower().endswith(('png', 'jpg', 'jpeg')):
        image_path = os.path.join(input_dir+"/images/", filename)
        data_aug.augment_image(image_path, output_dir)

# after cropping, the size of the images are (1080, 1671, 3)


### TRAINING THE MODEL

1. Load the Dataset

In [29]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import os

class ImageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 4])
        image = Image.open(img_name).convert("RGB")
        labels = self.data_frame.iloc[idx, 1:4].values
        labels = labels.astype('float').tolist()

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(labels)

# Dataset parameters
csv_file = 'data/single/single_img_regression_single_tip_pos.csv'
root_dir = 'data/single/augmented/images/'

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)), # image input size to model
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load dataset
dataset = ImageDataset(csv_file=csv_file, root_dir=root_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


2. Define the Model

In [30]:
import torch.nn as nn
import torchvision.models as models

# Load a pre-trained ResNet model
model = models.resnet18(pretrained=True)

# Replace the last fully connected layer to output 3 values (x3, y3, z3)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3) #output layers is 3-vector

# Move model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/Users/asltrunk/miniconda3/envs/trunk/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/asltrunk/miniconda3/envs/trunk/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/asltrunk/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100.0%


3. Train the model

In [31]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

print('Finished Training')
#took 5min on mac

Epoch [1/20], Loss: 0.4195
Epoch [2/20], Loss: 0.4380
Epoch [3/20], Loss: 0.0103
Epoch [4/20], Loss: 0.0100
Epoch [5/20], Loss: 0.0080
Epoch [6/20], Loss: 0.0052
Epoch [7/20], Loss: 0.0052
Epoch [8/20], Loss: 0.0038
Epoch [9/20], Loss: 0.0039
Epoch [10/20], Loss: 0.0029
Epoch [11/20], Loss: 0.0025
Epoch [12/20], Loss: 0.0022
Epoch [13/20], Loss: 0.0018
Epoch [14/20], Loss: 0.0017
Epoch [15/20], Loss: 0.0013
Epoch [16/20], Loss: 0.0013
Epoch [17/20], Loss: 0.0012
Epoch [18/20], Loss: 0.0017
Epoch [19/20], Loss: 0.0009
Epoch [20/20], Loss: 0.0023
Finished Training


4. Evaluate the model

In [32]:
# Example prediction on a single image
model.eval()  # Set the model to evaluation mode


# testin on training set (should be able to memorize training data as a first step)
with torch.no_grad():
    for i in range(5):  # Test on the first 5 images
        inputs, labels = dataset[i]
        inputs = inputs.unsqueeze(0).to(device)  # Add batch dimension and move to GPU
        outputs = model(inputs)
        print(f'Predicted: {outputs.cpu().numpy()}, Ground Truth: {labels.numpy()}')

Predicted: [[-0.60934645  0.10587105  0.18116018]], Ground Truth: [-0.1637994  -0.22002804 -0.01108289]
Predicted: [[-0.57156754  0.11285207  0.18355541]], Ground Truth: [-0.1637994  -0.22002804 -0.01108289]
Predicted: [[-0.24884418  0.09402524  0.22770816]], Ground Truth: [ 0.1755904  -0.21333098  0.01334643]
Predicted: [[-0.30379704  0.06469792  0.192056  ]], Ground Truth: [ 0.1755904  -0.21333098  0.01334643]
Predicted: [[-0.45932662  0.10767904  0.00098242]], Ground Truth: [-0.00449705 -0.18612087 -0.16756463]
